In [1]:
import os
import numpy as np
import polars as pl
from lets_plot import *
LetsPlot.setup_html()

In [2]:
df = pl.read_csv("data/decays.csv")

In [3]:
df.head()

2.80_10_left,2.80_10_right,2.80_10_all,2.80_20_left,2.80_20_right,2.80_20_all,2.80_40_left,2.80_40_right,2.80_40_all,2.80_60_left,2.80_60_right,2.80_60_all,3.00_10_left,3.00_10_right,3.00_10_all,3.00_20_left,3.00_20_right,3.00_20_all,3.00_40_left,3.00_40_right,3.00_40_all,3.00_60_left,3.00_60_right,3.00_60_all,3.50_10_left,3.50_10_right,3.50_10_all,3.50_20_left,3.50_20_right,3.50_20_all,3.50_40_left,3.50_40_right,3.50_40_all,3.50_60_left,3.50_60_right,3.50_60_all,4.00_10_left,4.00_10_right,4.00_10_all,4.00_20_left,4.00_20_right,4.00_20_all,4.00_40_left,4.00_40_right,4.00_40_all,4.00_60_left,4.00_60_right,4.00_60_all,timestep
i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
84,81,98,172,163,193,363,358,394,546,569,611,92,93,102,171,181,204,363,383,409,566,569,614,95,103,105,190,193,205,409,414,419,613,617,624,104,104,105,207,207,210,419,418,420,626,629,630,1
27,26,43,58,62,101,108,108,182,147,158,263,27,25,45,52,47,83,91,116,175,176,174,290,25,27,44,56,58,85,202,192,287,268,283,404,51,53,73,114,106,154,241,253,329,399,382,519,2
7,12,19,21,28,56,39,38,84,54,60,126,11,9,25,18,15,40,28,45,85,71,75,155,9,7,20,15,17,38,112,114,208,168,173,291,28,32,55,82,74,125,172,160,265,288,273,441,3
2,4,10,7,14,29,16,17,49,28,26,75,5,3,15,7,7,23,13,19,46,30,31,82,1,1,6,5,3,16,67,69,141,114,113,228,16,24,44,55,51,99,126,118,232,219,208,386,4
1,0,4,5,10,21,8,9,31,14,8,41,2,3,9,4,2,13,8,7,25,11,14,46,1,1,2,5,2,12,42,48,110,84,86,194,11,9,28,49,41,84,95,88,199,174,169,347,5


In [4]:
for col in df.columns:
    if col != "timestep":
        df = df.with_columns(
            ((pl.col(col)/pl.col(col).max())*100)
        ) 

In [5]:
df_melted = df.head(20).melt(id_vars="timestep",value_name='counts',variable_name="case")
df_melted.head()

C:\Users\zafi_\AppData\Local\Temp\ipykernel_10848\932811825.py:1: DeprecationWarning: `DataFrame.melt` is deprecated. Use `unpivot` instead, with `index` instead of `id_vars` and `on` instead of `value_vars`
  df_melted = df.head(20).melt(id_vars="timestep",value_name='counts',variable_name="case")


timestep,case,counts
i64,str,f64
1,"""2.80_10_left""",100.0
2,"""2.80_10_left""",32.142857
3,"""2.80_10_left""",8.333333
4,"""2.80_10_left""",2.380952
5,"""2.80_10_left""",1.190476


In [6]:
dfm = df_melted.with_columns(
    pl.col("case").str.split("_").list.get(0).cast(pl.Float64).alias("kt"),
    pl.col("case").str.split("_").list.get(1).cast(pl.UInt16).alias("um"),
    pl.col("case").str.split("_").list.get(2).alias("which"),
)

In [7]:
dfm

timestep,case,counts,kt,um,which
i64,str,f64,f64,u16,str
1,"""2.80_10_left""",100.0,2.8,10,"""left"""
2,"""2.80_10_left""",32.142857,2.8,10,"""left"""
3,"""2.80_10_left""",8.333333,2.8,10,"""left"""
4,"""2.80_10_left""",2.380952,2.8,10,"""left"""
5,"""2.80_10_left""",1.190476,2.8,10,"""left"""
…,…,…,…,…,…
16,"""4.00_60_all""",13.333333,4.0,60,"""all"""
17,"""4.00_60_all""",12.063492,4.0,60,"""all"""
18,"""4.00_60_all""",11.111111,4.0,60,"""all"""


In [8]:
dfm_whole = dfm.filter(pl.col("which") == "all")

In [9]:
dfm_whole

timestep,case,counts,kt,um,which
i64,str,f64,f64,u16,str
1,"""2.80_10_all""",100.0,2.8,10,"""all"""
2,"""2.80_10_all""",43.877551,2.8,10,"""all"""
3,"""2.80_10_all""",19.387755,2.8,10,"""all"""
4,"""2.80_10_all""",10.204082,2.8,10,"""all"""
5,"""2.80_10_all""",4.081633,2.8,10,"""all"""
…,…,…,…,…,…
16,"""4.00_60_all""",13.333333,4.0,60,"""all"""
17,"""4.00_60_all""",12.063492,4.0,60,"""all"""
18,"""4.00_60_all""",11.111111,4.0,60,"""all"""


In [10]:
kts = dfm_whole.select("kt").to_series().unique(maintain_order=True).to_list()

In [11]:
def decay_plot(df:pl.DataFrame):
    ums = dfm_whole.select("um").to_series().unique(maintain_order=True).to_list()
    kt = dfm_whole.select("um").to_series().to_list()[0]
    print(kt)
    kt = str(kt)
    kwargs = {
        "labels":[f"{x}µM" for x in ums],
    }
    p = (
        ggplot(df,aes(x='timestep',y='counts',color=as_discrete('um')))
        + geom_line(size=1.2)
        + geom_point(size=5,shape=21,color="#1f1f1f",mapping=aes(fill=as_discrete('um')))
        + scale_color_brewer(**kwargs)
        + scale_fill_brewer(**kwargs)
        + theme_classic()
        + labs(x= "Timestep (a.u)",y="Survival Fraction %")
        + theme(legend_title=element_blank(),
                text=element_text(family="Arial",color="#2f2f2f"),
        )
                
    )
    return p

In [12]:
decay_plot(dfm_whole.filter(pl.col("kt")==2.8))

10


In [13]:
gggrid(
    [decay_plot(dfm_whole.filter(pl.col("kt")==kt)) for kt in kts],
    ncol=2)

10
10
10
10


In [14]:
gggrid([decay_plot(dfm_whole.filter(pl.col("kt")==kt)) for kt in kts],ncol=2).to_png("decays.png")


10
10
10
10


'c:\\Users\\zafi_\\paper\\residence2\\6_FD\\decays.png'

In [15]:
gggrid([decay_plot(dfm_whole.filter(pl.col("kt")==kt)) for kt in kts],ncol=2).to_pdf("../Figures/SI-fig3.pdf")

10
10
10
10


'c:\\Users\\zafi_\\paper\\residence2\\Figures\\SI-fig4.pdf'